# Bronze (BigQuery) a partir de Parquet na STG (GCS)

Este notebook cria:
1) uma **External Table** apontando para os Parquets em `gs://.../stg/bps/` com **Hive partitioning** (`year=` e `ingest_date=` no path)
2) uma **View Bronze** com **renames**, **tipagens (casts)** e algumas normalizações mínimas.

> **Por que View?** External table precisa refletir o schema do Parquet. Para renomear e tipar de forma controlada sem copiar dados, o padrão é **External Table + View (bronze)**.


## 0) Parâmetros

Edite as variáveis abaixo conforme seu projeto.

In [1]:
#from dotenv import load_dotenv
#from pathlib import Path
#load_dotenv("../../.env")

In [2]:
import os
from google.cloud import bigquery

# ---- ajuste aqui ----
PROJECT_ID = os.environ.get("PROJECT_ID")
BRONZE_DATASET = os.environ.get("BRONZE_DATASET")

# External table (aponta pro GCS)
EXT_TABLE = os.environ.get("EXT_TABLE")

# View Bronze (renames + tipagens)
BRONZE_VIEW = os.environ.get("BRONZE_VIEW")

# GCS
BUCKET = os.environ.get("BUCKET")
STG_PREFIX = os.environ.get("STG_PREFIX")  # deve terminar com '/'

# Garante prefixo com barra
if STG_PREFIX and not STG_PREFIX.endswith('/'):
    STG_PREFIX += '/'

GCS_HIVE_PREFIX = f"gs://{BUCKET}/{STG_PREFIX}"  # ex.: gs://meu-bucket/stg/bps/
GCS_URIS = [f"gs://{BUCKET}/{STG_PREFIX}*"]

print("PROJECT_ID:", PROJECT_ID)
print("BUCKET:", BUCKET)
print("STG_PREFIX:", STG_PREFIX)
print("Dataset Bronze:", BRONZE_DATASET)
print("External Table:", f"{PROJECT_ID}.{BRONZE_DATASET}.{EXT_TABLE}")
print("Bronze View:", f"{PROJECT_ID}.{BRONZE_DATASET}.{BRONZE_VIEW}")
print("GCS hive prefix:", GCS_HIVE_PREFIX)
print("GCS uris:", GCS_URIS)

bq = bigquery.Client(project=PROJECT_ID)


c:\Users\victo\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


PROJECT_ID: rq-pharma-data-lab-26k9
BUCKET: rq-pharma-raw-rq-pharma-data-lab-26k9
STG_PREFIX: stg/bps/
Dataset Bronze: bronze_bps
External Table: rq-pharma-data-lab-26k9.bronze_bps.ext_bps
Bronze View: rq-pharma-data-lab-26k9.bronze_bps.view_bronze_bps
GCS hive prefix: gs://rq-pharma-raw-rq-pharma-data-lab-26k9/stg/bps/
GCS uris: ['gs://rq-pharma-raw-rq-pharma-data-lab-26k9/stg/bps/*']


## 1) Criar dataset (se não existir)

In [3]:
sql = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{BRONZE_DATASET}`
OPTIONS(location="US");
"""
print(sql)
bq.query(sql).result()
print("OK: dataset pronto")



CREATE SCHEMA IF NOT EXISTS `rq-pharma-data-lab-26k9.bronze_bps`
OPTIONS(location="US");

OK: dataset pronto


## 2) Criar External Table (STG Parquet)

A partição vem do caminho:
- `.../year=YYYY/ingest_date=YYYY-MM-DD/part-xxxxx.parquet`

E habilitamos `require_hive_partition_filter` para evitar leituras acidentais.

In [4]:
sql = f"""
CREATE OR REPLACE EXTERNAL TABLE `{PROJECT_ID}.{BRONZE_DATASET}.{EXT_TABLE}`
WITH PARTITION COLUMNS (
  year INT64,
  ingest_date DATE
)
OPTIONS (
  format = 'PARQUET',
  uris = {GCS_URIS},
  hive_partition_uri_prefix = '{GCS_HIVE_PREFIX}',
  require_hive_partition_filter = TRUE
);
"""
print(sql)
bq.query(sql).result()
print("OK: external table criada/atualizada")



CREATE OR REPLACE EXTERNAL TABLE `rq-pharma-data-lab-26k9.bronze_bps.ext_bps`
WITH PARTITION COLUMNS (
  year INT64,
  ingest_date DATE
)
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://rq-pharma-raw-rq-pharma-data-lab-26k9/stg/bps/*'],
  hive_partition_uri_prefix = 'gs://rq-pharma-raw-rq-pharma-data-lab-26k9/stg/bps/',
  require_hive_partition_filter = TRUE
);

OK: external table criada/atualizada


## 3) Criar View Bronze (renames + tipagens)

Abaixo um exemplo de **normalização mínima** (estilo Silver-light dentro do Bronze):
- `uf`: trim + upper, valida tamanho 2
- `ano_compra`, `qtd_itens_comprados`, `source_year`: `SAFE_CAST` para INT64
- `preco_unitario`, `preco_total`: `SAFE_CAST` para FLOAT64
- `compra_ts` / `insercao_ts`: tenta parse em formatos comuns
- `compra_date`: derivado de `compra_ts`

⚠️ Ajuste os formatos de timestamp e nomes de campos conforme seu CSV original, se necessário.

In [5]:
sql = f"""
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BRONZE_DATASET}.{BRONZE_VIEW}` AS
WITH base AS (
  SELECT
    -- Partições (do path)
    year,
    ingest_date,

    -- Campos originais (do Parquet)
    SAFE_CAST(ano_compra AS INT64)              AS ano_compra,
    nome_instituicao                           AS instituicao_nome,
    cnpj_instituicao                           AS instituicao_cnpj,
    municipio_instituicao                      AS instituicao_municipio,
    -- normaliza UF
    CASE
      WHEN LENGTH(TRIM(UPPER(uf))) = 2 THEN TRIM(UPPER(uf))
      ELSE NULL
    END                                        AS uf,

    compra                                     AS compra_raw,
    insercao                                   AS insercao_raw,

    codigo_br                                  AS codigo_br,
    descricao_catmat                           AS catmat_descricao,
    unidade_fornecimento                       AS unidade_fornecimento,
    generico                                   AS generico,
    anvisa                                     AS anvisa,
    modalidade_compra                          AS modalidade_compra,
    tipo_compra                                AS tipo_compra,
    capacidade                                 AS capacidade,
    unidade_medida                             AS unidade_medida,
    unidade_fornecimento_capacidade            AS unidade_fornecimento_capacidade,
    cnpj_fornecedor                            AS fornecedor_cnpj,
    fornecedor                                 AS fornecedor_nome,
    cnpj_fabricante                            AS fabricante_cnpj,
    fabricante                                 AS fabricante_nome,

    SAFE_CAST(qtd_itens_comprados AS INT64)     AS qtd_itens_comprados,
    SAFE_CAST(preco_unitario AS FLOAT64)        AS preco_unitario,
    SAFE_CAST(preco_total AS FLOAT64)           AS preco_total,

    SAFE_CAST(source_year AS INT64)             AS source_year,
    source_object                              AS source_object,
    load_ts_utc                                AS load_ts_utc
  FROM `{PROJECT_ID}.{BRONZE_DATASET}.{EXT_TABLE}`
  WHERE TRUE
)
, parsed AS (
  SELECT
    *
    , COALESCE(
        SAFE.PARSE_TIMESTAMP('%Y/%m/%d %H:%M:%E*S', compra_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', compra_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%S', compra_raw),
        SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M:%S', compra_raw),
        SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M', compra_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%d', compra_raw)
      ) AS compra_ts
    , COALESCE(
        SAFE.PARSE_TIMESTAMP('%Y/%m/%d %H:%M:%E*S', insercao_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', insercao_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%S', insercao_raw),
        SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M:%S', insercao_raw),
        SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M', insercao_raw),
        SAFE.PARSE_TIMESTAMP('%Y-%m-%d', insercao_raw)
      ) AS insercao_ts
  FROM base
)
SELECT
  year,
  ingest_date,
  ano_compra,
  instituicao_nome,
  instituicao_cnpj,
  instituicao_municipio,
  uf,
  compra_raw,
  insercao_raw,
  codigo_br,
  catmat_descricao,
  unidade_fornecimento,
  generico,
  anvisa,
  modalidade_compra,
  tipo_compra,
  capacidade,
  unidade_medida,
  unidade_fornecimento_capacidade,
  fornecedor_cnpj,
  fornecedor_nome,
  fabricante_cnpj,
  fabricante_nome,
  qtd_itens_comprados,
  preco_unitario,
  preco_total,
  source_year,
  source_object,
  load_ts_utc,
  compra_ts,
  insercao_ts,
  DATE(compra_ts) AS compra_date
FROM parsed;
"""
print(sql)
bq.query(sql).result()
print("OK: view bronze criada/atualizada")



CREATE OR REPLACE VIEW `rq-pharma-data-lab-26k9.bronze_bps.view_bronze_bps` AS
WITH base AS (
  SELECT
    -- Partições (do path)
    year,
    ingest_date,

    -- Campos originais (do Parquet)
    SAFE_CAST(ano_compra AS INT64)              AS ano_compra,
    nome_instituicao                           AS instituicao_nome,
    cnpj_instituicao                           AS instituicao_cnpj,
    municipio_instituicao                      AS instituicao_municipio,
    -- normaliza UF
    CASE
      WHEN LENGTH(TRIM(UPPER(uf))) = 2 THEN TRIM(UPPER(uf))
      ELSE NULL
    END                                        AS uf,

    compra                                     AS compra_raw,
    insercao                                   AS insercao_raw,

    codigo_br                                  AS codigo_br,
    descricao_catmat                           AS catmat_descricao,
    unidade_fornecimento                       AS unidade_fornecimento,
    generico                                

## 4) Teste rápido (obrigatório filtrar partições)

Se `require_hive_partition_filter=TRUE`, sempre filtre por `year` e/ou `ingest_date`.

## Opcional: criar uma Bronze *materializada* (tabela gerenciada)

Se você quiser performance máxima (e não ficar preso ao GCS), você pode criar uma tabela particionada/clustada a partir da View.
Isso **copia** os dados para o BigQuery (custo de storage), mas deixa as queries muito mais rápidas.


In [7]:
from google.cloud import storage
from google.cloud import bigquery
import os, re

bq = bigquery.Client(project=PROJECT_ID)
gcs = storage.Client(project=PROJECT_ID)

# ---- controle ----
MATERIALIZE = os.environ.get("MATERIALIZE_BRONZE", "0") == "1"  # set 1 para rodar
BRONZE_TABLE = os.environ.get("BRONZE_TABLE")

# filtros opcionais (deixe None para pegar tudo)
YEARS_FILTER = None  # ex.: set([2024])
INGEST_FILTER = None  # ex.: set(['2026-02-14'])

if not MATERIALIZE:
    print("MATERIALIZE_BRONZE=0 -> pulando materialização")
else:
    # 1) lista partições existentes via GCS
    bucket = gcs.bucket(BUCKET)
    blobs = gcs.list_blobs(bucket, prefix=STG_PREFIX)

    parts = set()
    for b in blobs:
        name = b.name
        if not name.endswith('.parquet'):
            continue
        my = re.search(r"year=(20\d{2})", name)
        mi = re.search(r"ingest_date=(\d{4}-\d{2}-\d{2})", name)
        if not my or not mi:
            continue
        y = int(my.group(1))
        d = mi.group(1)
        if YEARS_FILTER and y not in YEARS_FILTER:
            continue
        if INGEST_FILTER and d not in INGEST_FILTER:
            continue
        parts.add((y, d))

    parts = sorted(parts)
    if not parts:
        raise RuntimeError("Nenhuma partição encontrada em STG para materializar.")

    print(f"Partições encontradas: {len(parts)}")
    print("Primeiras 10:", parts[:10])

    target = f"{PROJECT_ID}.{BRONZE_DATASET}.{BRONZE_TABLE}"
    view = f"{PROJECT_ID}.{BRONZE_DATASET}.{BRONZE_VIEW}"

    # 2) cria tabela com a primeira partição (já carrega) — evita WHERE 1=0 sem filtro
    y0, d0 = parts[0]
    sql_create = f"""
    CREATE OR REPLACE TABLE `{target}`
    PARTITION BY compra_date
    CLUSTER BY uf, codigo_br AS
    SELECT *
    FROM `{view}`
    WHERE year = {y0} AND ingest_date = DATE '{d0}';
    """
    print(sql_create)
    bq.query(sql_create).result()
    print(f"OK: criada e carregada 1ª partição: year={y0}, ingest_date={d0}")

    # 3) insere as demais partições
    for y, d in parts[1:]:
        sql_ins = f"""
        INSERT INTO `{target}`
        SELECT *
        FROM `{view}`
        WHERE year = {y} AND ingest_date = DATE '{d}';
        """
        print(f"Inserindo: year={y}, ingest_date={d}")
        bq.query(sql_ins).result()

    print("OK: materialização incremental concluída em", target)


Partições encontradas: 2
Primeiras 10: [(2024, '2026-02-13'), (2025, '2026-02-13')]

    CREATE OR REPLACE TABLE `rq-pharma-data-lab-26k9.bronze_bps.fato_bps`
    PARTITION BY compra_date
    CLUSTER BY uf, codigo_br AS
    SELECT *
    FROM `rq-pharma-data-lab-26k9.bronze_bps.view_bronze_bps`
    WHERE year = 2024 AND ingest_date = DATE '2026-02-13';
    
OK: criada e carregada 1ª partição: year=2024, ingest_date=2026-02-13
Inserindo: year=2025, ingest_date=2026-02-13
OK: materialização incremental concluída em rq-pharma-data-lab-26k9.bronze_bps.fato_bps
